In [1]:
import sys
import pathlib
import pandas as pd
from pandas import DataFrame
import logging
from typing import List
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [2]:
def create_html_file(logger: logging.Logger,
                     path: pathlib.Path,
                     html_content: str
                     ) -> None:
    """
    Create an HTML file at the specified path.

    Parameters
    ----------
    logger : logging.Logger
        Logger instance for logging events.
    path : pathlib.Path
        Path to save the HTML file.
    html_content : str
        The HTML content to write to the file.

    Returns
    -------
    None
    """
    try:
        with open(path, "w") as file:
            file.write(html_content)
        logger.info(f"HTML file saved successfully: {path}")
    except Exception as e:
        logger.error(f"Failed to create HTML file: {e}")


def process_dataframe(df, group_by_column, agg_column, order=None, agg_func='sum'):
    """
    Groups, categorizes, and sorts a DataFrame based on the provided parameters.

    Parameters:
    ----------
    df : pd.DataFrame
        Input DataFrame.
    group_by_column : str
        Column name to group by.
    agg_column : str
        Column name to aggregate.
    order : list, optional
        Desired order for categorization (default is None).
    agg_func : str, optional
        Aggregation function (default is 'sum').

    Returns:
    -------
    pd.DataFrame
        Processed DataFrame.
    """
    grouped_df = df.groupby(group_by_column, as_index=False).agg({agg_column: agg_func})
    if order:
        grouped_df[group_by_column] = pd.Categorical(grouped_df[group_by_column], categories=order, ordered=True)
        grouped_df = grouped_df.sort_values(group_by_column)
    return grouped_df

def create_toggleable_box_plots(dataframes: list, x_columns: list, y_columns: list, titles: list, color: str = "#0033A0"):
    """
    Creates a toggleable Plotly figure with box plots for multiple X and Y columns.

    Parameters:
    -----------
    dataframes : list
        List of DataFrames for each box plot.
    x_columns : list
        List of column names for the x-axis.
    y_columns : list
        List of column names for the y-axis.
    titles : list
        List of titles for each box plot.
    color : str
        Hex code for the box plot color.

    Returns:
    --------
    fig : go.Figure
        Plotly figure object with toggleable box plots.
    """
    fig = go.Figure()

    for i, (df, x_column, y_column, title) in enumerate(zip(dataframes, x_columns, y_columns, titles)):
        box = go.Box(
            x=df[x_column],
            y=df[y_column],
            name=title,
            marker_color=color,
            visible=(i == 0)  # Show only the first plot initially
        )
        fig.add_trace(box)

    # Create buttons for toggling
    buttons = [
        dict(
            label=title,
            method="update",
            args=[
                {"visible": [i == j for j in range(len(y_columns))]},  # Toggle visibility
                {"title": title}  # Update the title dynamically
            ]
        )
        for i, title in enumerate(titles)
    ]

    # Add buttons to the layout
    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'x': 1.25,  # Center the menu horizontally
            'xanchor': 'center',
            'y': 1.15,  # Position the menu at the same height as the title
            'yanchor': 'top',
            'direction': 'down'
        }],
        title=titles[0],
        xaxis_title="Quarter",
        yaxis_title=y_columns[0],
        width=900,
        height=800,
        margin=dict(r=150)  # Add margin for the buttons
    )

    return fig


# Define the modified function with color palette support
def create_toggleable_pie_charts(dataframes: List[pd.DataFrame],
                                 labels_columns: List[str],
                                 values_columns: List[str],
                                 titles: List[str],
                                 color_palettes: List[List[str]],
                                 width: int = 900,
                                 height: int = 800) -> go.Figure:
    """
    Creates a Plotly figure with toggleable pie charts for multiple DataFrames, using custom color palettes.

    Parameters
    ----------
    dataframes : list of DataFrame
        List of DataFrames for pie charts.
    labels_columns : list of str
        List of column names for labels in each DataFrame.
    values_columns : list of str
        List of column names for values in each DataFrame.
    titles : list of str
        List of titles for each pie chart.
    color_palettes : list of list of str
        List of color palettes for each pie chart (one list of colors per chart).
    width : int, optional
        Width of the figure (default is 1000).
    height : int, optional
        Height of the figure (default is 800).

    Returns
    -------
    go.Figure
        The Plotly figure object with toggleable pie charts.
    """
    if not (len(dataframes) == len(labels_columns) == len(values_columns) == len(titles) == len(color_palettes)):
        raise ValueError("All input lists must have the same length.")

    fig = go.Figure()

    for i, (df, labels_column, values_column, palette) in enumerate(zip(dataframes, labels_columns, values_columns, color_palettes)):
        pie = go.Pie(
            labels=df[labels_column],
            values=df[values_column],
            name=titles[i],
            marker=dict(colors=palette),  # Use the custom color palette
            visible=(i == 0),  # Show only the first plot initially
            textinfo="percent",  # Show only the percentage
            insidetextorientation="horizontal",  # Display text inside slices horizontally
            textposition="inside",  # Force text inside slices
            hoverinfo="label+percent",  # Show label and percent on hover
            hole=0.4  # Scale down the pie chart slightly for better spacing
        )
        fig.add_trace(pie)

    buttons = [
        {'label': title,
         'method': 'update',
         'args': [{'visible': [i == j for j in range(len(dataframes))]},
                  {'title': title}]}
        for i, title in enumerate(titles)
    ]

    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'x': 1.25,  # Center the menu horizontally
            'xanchor': 'center',
            'y': 1.15,  # Position the menu at the same height as the title
            'yanchor': 'top',
            'direction': 'down'
        }],
        title=titles[0],
        width=width,
        height=height,
        margin=dict(l=50, r=300, t=100, b=50),  # Add more space to the right margin
        legend=dict(
            orientation="v",  # Vertical orientation
            x=1.3,  # Move legend further to the right
            y=0.5,
            xanchor="left",
            yanchor="middle",
            bgcolor="rgba(255, 255, 255, 0.8)"  # Add a semi-transparent background for clarity
        )
    )

    return fig



def create_toggleable_bar_charts(dataframes: List[pd.DataFrame],
                                 x_columns: List[str],
                                 y_columns: List[str],
                                 titles: List[str],
                                 colors: List[str] = None,
                                 single_color: str = "#0033A0",
                                 width: int = 900,
                                 height: int = 800) -> go.Figure:
    """
    Creates a Plotly figure with toggleable stacked or single bar charts for multiple DataFrames.

    Parameters
    ----------
    dataframes : list of DataFrame
        List of DataFrames for bar charts.
    x_columns : list of str
        List of column names for x-axis in each DataFrame.
    y_columns : list of str
        List of column names for y-axis in each DataFrame.
    titles : list of str
        List of titles for each bar chart.
    colors : list of str, optional
        List of hex color codes for the metrics in each stacked bar.
    single_color : str, optional
        Hex color code for single-metric bar charts (default is "#0033A0").
    width : int, optional
        Width of the figure (default is 800).
    height : int, optional
        Height of the figure (default is 600).

    Returns
    -------
    go.Figure
        The Plotly figure object with toggleable bar charts.
    """
    if not (len(dataframes) == len(x_columns) == len(y_columns) == len(titles)):
        raise ValueError("All input lists must have the same length.")

    fig = go.Figure()

    for i, df in enumerate(dataframes):
        if 'Metric' in df.columns and colors:  # Check for multiple metrics
            for metric in df['Metric'].unique():
                filtered_df = df[df['Metric'] == metric]
                bar = go.Bar(
                    x=filtered_df[x_columns[i]],
                    y=filtered_df[y_columns[i]],
                    name=f"{titles[i]} - {metric}",
                    marker=dict(color=colors[df['Metric'].unique().tolist().index(metric)])
                )
                fig.add_trace(bar)
        else:  # Single-metric case
            bar = go.Bar(
                x=df[x_columns[i]],
                y=df[y_columns[i]],
                name=titles[i],
                marker=dict(color=single_color)
            )
            fig.add_trace(bar)

    buttons = []
    trace_offset = 0

    for i, df in enumerate(dataframes):
        num_traces = len(df['Metric'].unique()) if 'Metric' in df.columns and colors else 1
        visibility = [False] * len(fig.data)
        for j in range(num_traces):
            visibility[trace_offset + j] = True

        buttons.append({
            'label': titles[i],
            'method': 'update',
            'args': [
                {'visible': visibility},
                {'title': titles[i]}
            ]
        })

        trace_offset += num_traces

    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'x': 1.5,  # Center the menu horizontally
            'xanchor': 'center',
            'y': 1.15,  # Position the menu at the same height as the title
            'yanchor': 'top',
            'direction': 'down'
        }],
        title=titles[0],
        width=900,
        height=800,
        barmode="stack" if colors else "group",
        margin=dict(r=150),  # Add margin to avoid overlap
        xaxis=dict(
            automargin=True,
            fixedrange=False,  # Allow scrolling on x-axis
            rangeslider=dict(visible=True)  # Add a range slider
        )
    )

    for i, trace in enumerate(fig.data):
        trace.visible = (i < (len(df['Metric'].unique()) if 'Metric' in df.columns and colors else 1))

    return fig


def create_toggleable_bar_chart_with_fixed_barmode(stacked_figs, titles, xaxis_titles, yaxis_titles,  non_stacked_figs=None):
    """
    Create a toggleable bar chart (stacked and non-stacked) from multiple bar charts with dynamic axis titles.
    Ensures that stacked charts remain stacked and non-stacked charts remain grouped.

    Parameters
    ----------
    stacked_figs : list of go.Figure
        List of stacked bar chart figures.
    non_stacked_figs : list of go.Figure
        List of non-stacked bar chart figures.
    titles : list of str
        List of chart titles corresponding to each figure.
    xaxis_titles : list of str
        List of x-axis titles corresponding to each figure.
    yaxis_titles : list of str
        List of y-axis titles corresponding to each figure.

    Returns
    -------
    go.Figure
        A Plotly figure object with toggleable bar charts.
    """
    fig = go.Figure()

    # Add traces from stacked bar chart figures
    for stacked_fig in stacked_figs:
        for trace in stacked_fig.data:
            fig.add_trace(trace)
    buttons = []
    trace_offset = 0

    # Handle visibility and buttons for stacked bar charts
    for i, stacked_fig in enumerate(stacked_figs):
        visibility = [False] * len(fig.data)
        for j in range(len(stacked_fig.data)):
            visibility[trace_offset + j] = True

        buttons.append({
            'label': titles[i],
            'method': 'update',
            'args': [
                {'visible': visibility},
                {'title': titles[i], 'xaxis.title': xaxis_titles[i], 'yaxis.title': yaxis_titles[i], 'barmode': 'stack'}
            ]
        })
        trace_offset += len(stacked_fig.data)

    # Update layout with buttons
    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'x': 1.15,
            'xanchor': 'center',
            'y': 1.15,
            'yanchor': 'top',
            'direction': 'down'
        }],
        title=titles[0],
        xaxis=dict(title=xaxis_titles[0], automargin=True),
        yaxis=dict(title=yaxis_titles[0]),
        width=900,
        height=800
    )

    # Set initial visibility for the first chart
    for i, trace in enumerate(fig.data):
        trace.visible = (i < len(stacked_figs[0].data) if stacked_figs else 0)

    return fig

def label_small_weights(df, weight_column, label_col ,threshold=0.01):
    """
    Transforms the company name to 'Other' if the weight is smaller than the threshold.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        weight_column (str): The name of the column containing weights.
        threshold (float): The weight threshold for transforming to 'Other' (default is 0.01 or 1%).

    Returns:
        pd.DataFrame: A DataFrame with updated company names.
    """
    # Transform the 'Company' column to 'Other' for rows where weight is smaller than the threshold
    df[label_col] = df.apply(lambda row: 'Other' if row[weight_column] < threshold else row[label_col], axis=1)
    return df

def group_rounds(df, round_column):
    """
    Groups similar investment rounds into broader categories, excluding Series A to J.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        round_column (str): The name of the column containing the investment rounds.

    Returns:
        pd.DataFrame: A DataFrame with a new column 'Round Grouped' containing the broader categories.
    """
    # Define the grouping of similar rounds, excluding Series A to J
    round_mapping = {
        'Seed': 'Seed Stage',
        'Pre-seed': 'Seed Stage',
        'Seed Bridge': 'Seed Stage',
        'Late-seed': 'Seed Stage',
        'Bridge to Series B': 'Bridge Rounds',
        'Series A Bridge': 'Bridge Rounds',
        'Bridge': 'Bridge Rounds',
        'Pre-Seed': 'Seed Stage',
        'Common': 'Other',
        'Other': 'Other'
    }

    # Apply the mapping to create a new column
    df[round_column] = df[round_column].map(round_mapping).fillna(df[round_column])

    return df


def create_toggleable_bar_charts(dataframes: List[pd.DataFrame],
                                 x_columns: List[str],
                                 y_columns: List[str],
                                 titles: List[str],
                                 colors: List[str] = None,
                                 single_color: str = "#0033A0",
                                 width: int = 900,
                                 height: int = 800,
                                 orientation: str = "h") -> go.Figure:
    """
    Creates a Plotly figure with toggleable stacked or single bar charts for multiple DataFrames.

    Parameters
    ----------
    dataframes : list of DataFrame
        List of DataFrames for bar charts.
    x_columns : list of str
        List of column names for x-axis in each DataFrame.
    y_columns : list of str
        List of column names for y-axis in each DataFrame.
    titles : list of str
        List of titles for each bar chart.
    colors : list of str, optional
        List of hex color codes for the metrics in each stacked bar.
    single_color : str, optional
        Hex color code for single-metric bar charts (default is "#0033A0").
    width : int, optional
        Width of the figure (default is 800).
    height : int, optional
        Height of the figure (default is 600).
    orientation : str, optional
        Orientation of the bar charts, either 'v' for vertical or 'h' for horizontal (default is 'v').

    Returns
    -------
    go.Figure
        The Plotly figure object with toggleable bar charts.
    """
    if not (len(dataframes) == len(x_columns) == len(y_columns) == len(titles)):
        raise ValueError("All input lists must have the same length.")

    fig = go.Figure()

    for i, df in enumerate(dataframes):
        if 'Metric' in df.columns and colors:  # Check for multiple metrics
            for metric in df['Metric'].unique():
                filtered_df = df[df['Metric'] == metric]
                bar = go.Bar(
                    x=filtered_df[x_columns[i]] if orientation == "v" else filtered_df[y_columns[i]],
                    y=filtered_df[y_columns[i]] if orientation == "v" else filtered_df[x_columns[i]],
                    name=f"{titles[i]} - {metric}",
                    marker=dict(color=colors[df['Metric'].unique().tolist().index(metric)]),
                    orientation=orientation
                )
                fig.add_trace(bar)
        else:  # Single-metric case
            bar = go.Bar(
                x=df[x_columns[i]] if orientation == "v" else df[y_columns[i]],
                y=df[y_columns[i]] if orientation == "v" else df[x_columns[i]],
                name=titles[i],
                marker=dict(color=single_color),
                orientation=orientation
            )
            fig.add_trace(bar)

    buttons = []
    trace_offset = 0

    for i, df in enumerate(dataframes):
        num_traces = len(df['Metric'].unique()) if 'Metric' in df.columns and colors else 1
        visibility = [False] * len(fig.data)
        for j in range(num_traces):
            visibility[trace_offset + j] = True

        buttons.append({
            'label': titles[i],
            'method': 'update',
            'args': [
                {'visible': visibility},
                {'title': titles[i]}
            ]
        })

        trace_offset += num_traces

    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'x': 1.5,  # Center the menu horizontally
            'xanchor': 'center',
            'y': 1.15,  # Position the menu at the same height as the title
            'yanchor': 'top',
            'direction': 'down'
        }],
        title=titles[0],
        width=900,
        height=800,
        barmode="stack" if colors else "group",
        margin=dict(r=150),  # Add margin to avoid overlap
        xaxis=dict(
            automargin=True,
            fixedrange=False,  # Allow scrolling on x-axis
            rangeslider=dict(visible=True)  # Add a range slider
        ),
    )

    for i, trace in enumerate(fig.data):
        trace.visible = (i < (len(df['Metric'].unique()) if 'Metric' in df.columns and colors else 1))

    return fig


def create_toggleable_bar_chart_with_dynamic_barmode(stacked_figs, titles, xaxis_titles, yaxis_titles, bar_modes, non_stacked_figs=None):
    """
    Create a toggleable bar chart (stacked and/or grouped) from multiple bar charts with dynamic axis titles.
    Allows specifying different bar modes (e.g., 'stack' or 'group') for each chart.

    Parameters
    ----------
    stacked_figs : list of go.Figure
        List of bar chart figures (stacked or grouped).
    titles : list of str
        List of chart titles corresponding to each figure.
    xaxis_titles : list of str
        List of x-axis titles corresponding to each figure.
    yaxis_titles : list of str
        List of y-axis titles corresponding to each figure.
    bar_modes : list of str
        List of bar modes (e.g., 'stack', 'group') corresponding to each figure.
    non_stacked_figs : list of go.Figure, optional
        List of additional bar chart figures (e.g., for non-stacked data).

    Returns
    -------
    go.Figure
        A Plotly figure object with toggleable bar charts and dynamic bar modes.
    """
    import plotly.graph_objects as go

    fig = go.Figure()

    # Add traces from stacked or grouped bar chart figures
    for stacked_fig in stacked_figs:
        for trace in stacked_fig.data:
            fig.add_trace(trace)

    buttons = []
    trace_offset = 0

    # Handle visibility and buttons for bar charts
    for i, stacked_fig in enumerate(stacked_figs):
        visibility = [False] * len(fig.data)
        for j in range(len(stacked_fig.data)):
            visibility[trace_offset + j] = True

        buttons.append({
            'label': titles[i],
            'method': 'update',
            'args': [
                {'visible': visibility},
                {
                    'title': titles[i],
                    'xaxis.title': xaxis_titles[i],
                    'yaxis.title': yaxis_titles[i],
                    'barmode': bar_modes[i]  # Use dynamic bar mode
                }
            ]
        })
        trace_offset += len(stacked_fig.data)

    # Update layout with buttons
    fig.update_layout(
        updatemenus=[{
            'buttons': buttons,
            'x': 1.15,
            'xanchor': 'center',
            'y': 1.15,
            'yanchor': 'top',
            'direction': 'down'
        }],
        title=titles[0],
        xaxis=dict(title=xaxis_titles[0], automargin=True),
        yaxis=dict(title=yaxis_titles[0]),
        width=900,
        height=800
    )

    # Set initial visibility for the first chart
    for i, trace in enumerate(fig.data):
        trace.visible = (i < len(stacked_figs[0].data) if stacked_figs else 0)

    return fig


# Function to prepare data for stacked bar charts
def prepare_stacked_data(df, group_by_col, value_col, metric_name):
    """
    Prepare data for stacked bar chart by renaming and assigning metric labels.
    """
    return process_dataframe(df, group_by_col, value_col).rename(columns={value_col: "Value"}).assign(Metric=metric_name)


def calculate_weights(df, group_by_cols, total_nav_df, order):
    """
    Calculate weights and sort data by Report and Weight.
    """
    grouped_df = df.groupby(group_by_cols, as_index=False).agg({'Calculate Nav Eur': 'sum'})
    merged_df = grouped_df.merge(total_nav_df, on='Report', suffixes=('', '_Total'))
    merged_df['Weight'] = merged_df['Calculate Nav Eur'] / merged_df['Calculate Nav Eur_Total']
    merged_df['Report'] = pd.Categorical(merged_df['Report'], categories=order, ordered=True)
    return merged_df.sort_values(['Report', 'Weight'], ascending=[True, False])

In [3]:
path = pathlib.Path(r'C:\Users\juann\OneDrive\Escritorio')
nav_study_case = path / 'Case_Study_Final_data.csv'
df = pd.read_csv(nav_study_case)
df.columns = df.columns.str.strip()
df.columns = df.columns.str.title()

bbva_colors = ["#0033A0", "#66A5D9", "#00509E", "#002A5C", "#0074D9", "#4682B4", "#00A9CE", "#D9D9D6", "#F4F4F4", "#4D4D4F", "#008080", "#F5F5DC", "#001F54", "#004080", "#0066FF", "#40E0D0"]
order = ["4Q23", "1Q24", "2Q24", "3Q24"]


key_columns = ['Manager', 'Fund', 'Vertical', 'Report', 'Main Operating Country By Revenues (Country)']

# Ensure all columns are strings and concatenate them to form a unique key
df['Unique_ID'] = df[key_columns].astype(str).agg('-'.join, axis=1)

# Hash the concatenated string to create a compact unique ID
df['Unique_ID'] = df['Unique_ID'].apply(hash)

# Create a new dataframe containing unique_ID
unique_rows = df.groupby('Unique_ID').first().reset_index()

In [4]:
last_q = df[df['Report'] == '3Q24']
last_q = last_q.loc[lambda x: (x['Calculate Nav Eur'] > 0) & (x['Calculate Investment Eur'] > 0)]
# Define the reversed dark blue color scale
dark_blue_scale_reversed = [
    [0.0, "#007bff"],  # Bright blue for lower values
    [0.2, "#0066cc"],
    [0.4, "#00509e"],
    [0.6, "#004080"],
    [0.8, "#003366"],
    [1.0, "#001f3f"],  # Very dark blue for higher values
]

# Aggregate data for Investment EUR and PnL EUR by country
country_agg_data = last_q.groupby("Main Operating Country By Revenues (Country)", as_index=False).agg({
    "Calculate Investment Eur": "sum",  # Replace with actual column name for Investment EUR
    "Pnl": "sum"  # Replace with actual column name for PnL EUR
})

# Add a column for Total NAV (sum of Investment EUR and PnL EUR)
country_agg_data["Total NAV"] = country_agg_data["Calculate Investment Eur"] + country_agg_data["Pnl"]

# Rename columns for better readability
country_agg_data = country_agg_data.rename(columns={
    "Main Operating Country By Revenues (Country)": "Country",
    "Calculate Investment Eur": "Investment EUR",
    "Pnl": "PnL EUR"
})

# Calculate the unique investment count per country
unique_investment_count = unique_rows.groupby(
    "Main Operating Country By Revenues (Country)", as_index=False
).size()
unique_investment_count = unique_investment_count.rename(columns={"size": "Unique Investment Count"})

# Merge the unique investment count into the aggregated data
country_agg_data = country_agg_data.merge(
    unique_investment_count,
    how="left",
    left_on="Country",
    right_on="Main Operating Country By Revenues (Country)"
)

# Add preformatted hover columns
country_agg_data["Investment EUR (M)"] = (country_agg_data["Investment EUR"] / 1e6).map("{:,.2f}M".format)
country_agg_data["PnL EUR (M)"] = (country_agg_data["PnL EUR"] / 1e6).map("{:,.2f}M".format)
country_agg_data["Total NAV (M)"] = (country_agg_data["Total NAV"] / 1e6).map("{:,.2f}M".format)

# Create a choropleth map
fig = px.choropleth(
    country_agg_data,
    locations="Country",
    locationmode="country names",
    color="Investment EUR",  # Use Investment EUR for country fill color
    hover_name="Country",
    hover_data={
        "Investment EUR": False,  # Hide raw Investment EUR
        "Investment EUR (M)": True,  # Preformatted column
        "PnL EUR (M)": True,  # Preformatted column
        "Total NAV (M)": True,  # Preformatted column
        "Unique Investment Count": True  # Include unique investment count in hover data
    },
    color_continuous_scale=dark_blue_scale_reversed,  # Custom dark blue scale
    title="Global Investment Portfolio: 3Q2024"
)

# Update layout for better visualization with increased size
fig.update_geos(
    showcoastlines=True,
    coastlinecolor="LightGray",
    showland=True,
    landcolor="whitesmoke",
    showlakes=True,
    lakecolor="LightBlue",
    projection_type="natural earth",
    showframe=False
)

fig.update_layout(
    width=1200,  # Increased width
    height=800,  # Increased height
    coloraxis_colorbar=dict(
        title="Investment (EUR)",
        ticks="outside",
        len=0.5,
        thickness=15
    )
)

# Render the map
geo_map = fig.to_html(full_html=False, include_plotlyjs="cdn")

In [5]:
# Data preparation for NAV metrics by continent
stacked_data_continent = pd.concat([
    prepare_stacked_data(df, 'Main Operating Country By Revenues (Continent)', 'Calculate Investment Eur', "Investment EUR"),
    prepare_stacked_data(df, 'Main Operating Country By Revenues (Continent)', 'Pnl', "PnL EUR")
])

# Stacked bar chart for NAV metrics by continent
nav_per_continent = px.bar(
    stacked_data_continent,
    x="Main Operating Country By Revenues (Continent)",
    y="Value",
    color="Metric",
    title="Overall NAV Metrics by Continent",
    labels={"Value": "EUR Value", "Main Operating Country By Revenues (Continent)": "Continent"},
    color_discrete_sequence=["#0033A0", "#66A5D9"],
    width=900,
    height=800
).to_html(full_html=False, include_plotlyjs="cdn")

In [6]:
# Non-stacked bar chart for "Number of Shares by Quarter"
# Group to get total NAV per quarter
total_nav_df = process_dataframe(df, group_by_column='Report', agg_column='Calculate Nav Eur')

# Calculate NAV weights for each category
nav_company_q = calculate_weights(df, ['Report', 'Company'], total_nav_df, order)
nav_company_q = label_small_weights(nav_company_q, weight_column='Weight', label_col='Company')
nav_round_q = calculate_weights(df, ['Report', 'Round'], total_nav_df, order)
nav_round_q = group_rounds(nav_round_q, round_column='Round')
nav_fund_q = calculate_weights(df, ['Report', 'Fund'], total_nav_df, order)
nav_fund_q = label_small_weights(nav_fund_q, weight_column='Weight', label_col='Fund')
nav_sector_q = calculate_weights(df, ['Report', 'Sector'], total_nav_df, order)
nav_sector_q = label_small_weights(nav_sector_q, weight_column='Weight', label_col='Sector')

In [7]:
# Data preparation for NAV metrics by quarter
stacked_data_quarter = pd.concat([
    prepare_stacked_data(df, 'Report', 'Calculate Investment Eur', "Investment EUR"),
    prepare_stacked_data(df, 'Report', 'Pnl', "PnL EUR")
])
stacked_data_quarter['Report'] = pd.Categorical(stacked_data_quarter['Report'], categories=order, ordered=True)
stacked_data_quarter = stacked_data_quarter.sort_values(by='Report').reset_index(drop=True)
# Stacked bar chart for NAV metrics by quarter
stacked_data_quarter_fig = px.bar(
    stacked_data_quarter,
    x="Report",
    y="Value",
    color="Metric",
    title="Overall NAV Metrics per Quarter",
    labels={"Value": "EUR Value", "Report": "Quarter"},
    color_discrete_sequence=["#0033A0", "#66A5D9"]
)

# Create stacked bar charts
stacked_company_fig = px.bar(
    nav_company_q,
    barmode='group',
    x="Report",
    y="Weight",
    color="Company",
    title="Overall NAV Metrics per Quarter (Companies)",
    labels={"Weight": "NAV Weight", "Report": "Quarter"},
    color_discrete_sequence=bbva_colors
)

stacked_round_fig = px.bar(
    nav_round_q,
    x="Report",
    y="Weight",
    color="Round",
    title="Overall NAV Metrics per Quarter (Rounds)",
    labels={"Weight": "NAV Weight", "Report": "Quarter"},
    color_discrete_sequence=bbva_colors
)

stacked_fund_fig = px.bar(
    nav_fund_q,
    x="Report",
    y="Weight",
    color="Fund",
    title="Overall NAV Metrics per Quarter (Funds)",
    labels={"Weight": "NAV Weight", "Report": "Quarter"},
    color_discrete_sequence=bbva_colors
)

stacked_sector_fig = px.bar(
    nav_sector_q,
    x="Report",
    y="Weight",
    color="Sector",
    title="Overall NAV Metrics per Quarter (Sector)",
    labels={"Weight": "NAV Weight", "Report": "Quarter"},
    color_discrete_sequence=bbva_colors
)

# Update titles and barmodes
titles = [
    "Overall NAV Metrics per Quarter",
    "Overall NAV Metrics per Quarter (Companies)",
    "Overall NAV Metrics per Quarter (Rounds)",
    "Overall NAV Metrics per Quarter (Funds)",
    "Overall NAV Metrics per Quarter (Sector)",
]
xaxis_titles = ["Quarter", "Quarter", "Quarter", "Quarter", "Quarter"]
yaxis_titles = ["EUR Value", "NAV Weight", "NAV Weight", "NAV Weight", "NAV Weight"]

# Create toggleable chart
# Mock the list of figures and parameters based on your data
stacked_figs = [
    stacked_data_quarter_fig,  # Stacked bar chart for overall NAV metrics per quarter
    stacked_company_fig,       # Grouped bar chart for companies
    stacked_round_fig,         # Grouped bar chart for rounds
    stacked_fund_fig,          # Grouped bar chart for funds
    stacked_sector_fig         # Grouped bar chart for sectors
]

# Define bar modes for each figure
bar_modes = ['group', 'stack', 'stack', 'stack', 'stack']

# Define titles, x-axis titles, and y-axis titles for each figure
titles = [
    "Overall NAV Metrics per Quarter",
    "Overall NAV Metrics per Quarter (Companies)",
    "Overall NAV Metrics per Quarter (Rounds)",
    "Overall NAV Metrics per Quarter (Funds)",
    "Overall NAV Metrics per Quarter (Sector)"
]

xaxis_titles = ["Quarter", "Quarter", "Quarter", "Quarter", "Quarter"]
yaxis_titles = ["EUR Value", "NAV Weight", "NAV Weight", "NAV Weight", "NAV Weight"]

# Call the function
toggleable_chart = create_toggleable_bar_chart_with_dynamic_barmode(
    stacked_figs=stacked_figs,
    titles=titles,
    xaxis_titles=xaxis_titles,
    yaxis_titles=yaxis_titles,
    bar_modes=bar_modes
)
# Render the chart
overall_nav_metrics = toggleable_chart.to_html(full_html=False, include_plotlyjs="cdn")

In [8]:
# Create the toggleable box plots with separate x_columns
fig = create_toggleable_box_plots(
    dataframes=[df, df],
    x_columns=["Report", "Report"],
    y_columns=["Invest Diff Eur", "Nav Diff Eur"],
    titles=["Distribution of Investment Differences (EUR)", "Distribution of NAV Differences (EUR)"],
    color="#0033A0"
)

# Show the plot
differences_metrics_overview = fig.to_html(full_html=False, include_plotlyjs="cdn")

In [9]:
# Using the 4q
tech_overview = process_dataframe(unique_rows, 'Technology', 'Company', agg_func='count')
customer_relationship = process_dataframe(unique_rows, 'Customer Relationship Model', 'Company', agg_func='count')
business_revenue = process_dataframe(unique_rows, 'Business/Revenue Model', 'Company', agg_func='count')

bbva_colors = ["#0033A0", "#66A5D9", "#00509E", "#002A5C", "#0074D9", "#4682B4", "#00A9CE", "#D9D9D6", "#F4F4F4", "#4D4D4F", "#008080", "#F5F5DC", "#001F54", "#004080", "#0066FF", "#40E0D0"]

industry_overview = create_toggleable_pie_charts([tech_overview, customer_relationship, business_revenue],
                             ['Technology', 'Customer Relationship Model', 'Business/Revenue Model'],
                             ['Company', 'Company', 'Company'],
                             ['Technology Overview', 'Customer Relationship Model Overview', 'Business/Revenue Model Overview'],
                                 color_palettes=[
        bbva_colors * len(tech_overview),  # Repeat color for each entry in tech_overview
        bbva_colors * len(customer_relationship),  # Repeat for customer_relationship
        bbva_colors * len(business_revenue)  # Repeat for business_revenue
    ]).to_html(full_html=False, include_plotlyjs="cdn")

In [10]:
#Investment overview
grouping = {
    "Equity": "Equity Instruments",
    "Preferred equity": "Equity Instruments",
    "Preferred Equity + Contingency commitment": "Equity Instruments",
    "Convertible Note": "Debt-Like Instruments",
    "Convertible bond": "Debt-Like Instruments",
    "Equity and shareholder loan": "Debt-Like Instruments",
    "SAFE (post money)": "Hybrid Instruments",
    "CLA": "Hybrid Instruments",
    "Prefunding loan and preferred equity": "Hybrid Instruments",
    "Other": "Other Instruments"
}


investment_type_overview = process_dataframe(unique_rows, 'Investment Type', 'Company', agg_func='count')
investment_instrument_overview = process_dataframe(unique_rows, 'Investment Instrument', 'Company', agg_func='count')
investment_instrument_overview["Investment Instrument"] = investment_instrument_overview["Investment Instrument"].map(grouping)
investment_overview = create_toggleable_pie_charts([investment_type_overview, investment_instrument_overview],
                             ['Investment Type', 'Investment Instrument'],
                             ['Company', 'Company'],
                             ['Investment Type Overview', 'Investment Instrument Overview'],
                                 color_palettes=[
        bbva_colors * len(investment_type_overview),  # Repeat color for each entry in tech_overview
        bbva_colors * len(investment_instrument_overview),  # Repeat for customer_relationship
    ]).to_html(full_html=False, include_plotlyjs="cdn")

In [11]:
#Distribution of Funds by Sector, Fund and Manager
# Sector investment distribution sorted by frequency
sector_invesment_distribution = unique_rows['Sector'].value_counts().reset_index()
sector_invesment_distribution.columns = ['Sector', 'Frequency']
sector_invesment_distribution = sector_invesment_distribution.sort_values(by='Sector', ascending=False)

# Vertical funds distribution sorted by frequency
vertical_funds_distribution = unique_rows['Vertical'].value_counts().reset_index()
vertical_funds_distribution.columns = ['Vertical', 'Frequency']
vertical_funds_distribution = vertical_funds_distribution.sort_values(by='Vertical', ascending=False)

# Manager per fund distribution sorted by frequency
manager_per_fund_distribution = unique_rows['Manager'].value_counts().reset_index()
manager_per_fund_distribution.columns = ['Manager', 'Frequency']
manager_per_fund_distribution = manager_per_fund_distribution.sort_values(by='Manager', ascending=False)
sector_manager_overview = create_toggleable_bar_charts([sector_invesment_distribution, vertical_funds_distribution, manager_per_fund_distribution],
                             ['Sector', 'Vertical', 'Manager'],
                             ['Frequency', 'Frequency', 'Frequency'],
                             ['Sector Investment Distribution', 'Vertical Fund Distribution', 'Management Distribution'],
                             colors=["#0033A0"]).to_html(full_html=False, include_plotlyjs="cdn")

In [12]:
df['Fund_company'] = df['Fund'] + ';' + df['Company']

In [13]:
fund_company_return = df.loc[lambda x: (x['Calculate Nav Eur'] > 0) & (x['Calculate Investment Eur'] > 0)].groupby(['Report', 'Fund_company', 'Sector'], as_index=False).agg({'Calculate Investment Eur':'sum', 
                                               'Calculate Nav Eur':'sum'}).dropna()

fund_company_return['Percentage Return'] = ((fund_company_return['Calculate Nav Eur'] - fund_company_return['Calculate Investment Eur']) / fund_company_return['Calculate Investment Eur']) * 100

In [14]:
fund_company_return['Report'] = pd.Categorical(fund_company_return['Report'], categories=order, ordered=True)

# Add a Size column to the dataframe
fund_company_return["Size"] = fund_company_return["Percentage Return"].apply(lambda x: max(x, 0.1))  # Replace non-positive values with 0.1

# Adding a column for coloring based on the Percentage Return
fund_company_return["Color"] = fund_company_return["Percentage Return"].apply(lambda x: "Negative" if x < 0 else "Positive")

In [15]:
import plotly.graph_objects as go

# Splitting the data into two dataframes based on positive and negative returns
positive_df = fund_company_return[fund_company_return["Percentage Return"] >= 0]
negative_df = fund_company_return[fund_company_return["Percentage Return"] < 0]

positive_df['Report'] = pd.Categorical(positive_df['Report'], categories=order, ordered=True)
positive_df.sort_values(['Report'], inplace=True)
negative_df['Report'] = pd.Categorical(negative_df['Report'], categories=order, ordered=True)
negative_df.sort_values(['Report'], inplace=True)
# Creating the bubble chart for positive returns
fig_positive = px.scatter(
    positive_df,
    x="Report",
    y="Percentage Return",
    size="Size",  # Bubble size
    color_discrete_sequence=["blue"],  # Set color for positive returns
    hover_data={"Fund_company": True, "Size": False, "Sector": True},  # Exclude 'Size' from hover
    title="Top Returns",
    labels={"Percentage Return": "Percentage Return (%)", "Report": "Quarter"}
).to_html(full_html=False, include_plotlyjs="cdn")

# Creating the bubble chart for negative returns
fig_negative = px.scatter(
    negative_df,
    x="Report",
    y="Percentage Return",
    size="Size",  # Bubble size
    color_discrete_sequence=["red"],  # Set color for negative returns
    hover_data={"Fund_company": True, "Size": False, "Sector": True},  # Exclude 'Size' from hover
    title="Bottom Returns",
    labels={"Percentage Return": "Percentage Return (%)", "Report": "Quarter"}
).to_html(full_html=False, include_plotlyjs="cdn")

In [16]:
# Data preparation
investment_type_data = pd.concat([
    process_dataframe(df, 'Investment Type', 'Calculate Investment Eur').rename(columns={"Calculate Investment Eur": "Value"}).assign(Metric="Investment EUR"),
    process_dataframe(df, 'Investment Type', 'Pnl').rename(columns={"Pnl": "Value"}).assign(Metric="PnL")
])

investment_instrument_data = pd.concat([
    process_dataframe(df, 'Investment Instrument', 'Calculate Investment Eur').rename(columns={"Calculate Investment Eur": "Value"}).assign(Metric="Investment EUR"),
    process_dataframe(df, 'Investment Instrument', 'Pnl').rename(columns={"Pnl": "Value"}).assign(Metric="PnL")
])

fund_data = pd.concat([
    process_dataframe(df, 'Fund', 'Calculate Investment Eur').rename(columns={"Calculate Investment Eur": "Value"}).assign(Metric="Investment EUR"),
    process_dataframe(df, 'Fund', 'Pnl').rename(columns={"Pnl": "Value"}).assign(Metric="PnL")
])

sector_data = pd.concat([
    process_dataframe(df, 'Sector', 'Calculate Investment Eur').rename(columns={"Calculate Investment Eur": "Value"}).assign(Metric="Investment EUR"),
    process_dataframe(df, 'Sector', 'Pnl').rename(columns={"Pnl": "Value"}).assign(Metric="PnL")
])

# Create toggleable stacked bar charts
pnl_overview = create_toggleable_bar_charts(
    dataframes=[investment_type_data, investment_instrument_data, fund_data, sector_data],
    x_columns=["Investment Type", "Investment Instrument", "Fund", "Sector"],
    y_columns=["Value", "Value", "Value", "Value"],
    titles=["NAV by Investment Type", "NAV by Investment Instrument", "NAV by Fund", "NAV by Sector"],
    colors=["#0033A0", "#66A5D9"],
    width=900,
    height=800
).to_html(full_html=False, include_plotlyjs="cdn")

In [17]:
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>BBVA Navbar and Case Study</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            margin: 0;
            font-family: Arial, sans-serif;
        }}
        .navbar {{
            background-color: #001F54;
            color: white;
            padding: 10px 20px;
            height: 60px;
            width: 100%;
            box-sizing: border-box;
        }}
        .navbar .content {{
            display: flex;
            justify-content: space-between;
            align-items: center;
        }}
        .navbar .content .logo {{
            font-size: 24px;
            font-weight: bold;
        }}
        .navbar .content .menu {{
            display: flex;
            gap: 20px;
        }}
        .navbar .content .menu a {{
            color: white;
            text-decoration: none;
            font-size: 16px;
            font-weight: 600;
        }}
        .navbar .content .menu a:hover {{
            text-decoration: underline;
        }}
        .main-content {{
            max-width: 1200px;
            margin: 20px auto;
            padding: 20px;
            font-size: 16px;
            line-height: 1.6;
        }}
        .main-content h1 {{
            font-size: 24px;
            margin-bottom: 20px;
            color: #0033A0;
        }}
        .results-container {{
            display: flex;
            flex-wrap: nowrap; /* Prevent wrapping */
            gap: 10px; /* Reduced gap between chart and text */
            align-items: flex-start;
            margin-bottom: 80px;
        }}
        .chart-container {{
            flex: 1 1 auto;
            display: flex;
            justify-content: center;
            align-items: center;
        }}
        .text-container {{
            flex: 0 1 auto; /* Allow text to shrink if needed */
            max-width: 40%; /* Keep the text from being too wide */
            margin-right: 10px; /* Consistent margin on the right */
            font-size: 16px;
            line-height: 1.6;
        }}
    </style>
</head>
<body>
    <div class="navbar">
        <div class="content">
            <div class="logo">Bilbao Vizcaya Investments (a BBVA Group company)</div>
        </div>
    </div>
    <div class="main-content">
        <h1>Case Study: What is driving my NAV?</h1>
        <p>BizCo, a company dedicated to managing venture capital investments, is recently having some issues. Following the exit of all the members of the Executive Committee (ExCo), who left the company to join KKR, the Board of Directors (BoD) of BizCo appointed a new ExCo composed of Santi, Ainhoa, and Unai. It is January 10th, 2025.</p>
        <p>The BoD needs to understand the dynamics of the Net Asset Value (NAV) of the portfolio, how this NAV evolved during 2024, and see if there are some estimates on what the NAV of the portfolio might look like as of the end of 2024 and as of today. In addition to this, it is key to understand what the underlying drivers are that explain the NAV and Money On Invested Capital (MOIC) dynamics of the portfolio, and the different clusters explaining it.</p>
        <p>The ExCo is new to the role and needs to carry out this analysis for the meeting scheduled with the BoD on Friday 24th, 2025. The ExCo calls you, with a background in Data Analysis, to help them out with this task and provides you with a spreadsheet with the information of the portfolio at the portfolio company level (see attached) and some public market data.</p>
        <p>The spreadsheet has different qualitative fields (e.g., report date, report currency, manager, fund, vertical, company name, technology, business model, investment status, round, current round, investment instrument) and quantitative fields (e.g., #shares, Last Price / Share, NAV, Investment). The info is at the investment level, so there might be many investments carried out on the same company at different points in time and valuations. Also, there are some listed companies on public markets among the portfolio companies.</p>
        <p>You need to decide the type of analysis and outputs you will build in order to meet the needs of BizCo’s BoD. You are free to ask as many questions as required, that will be responded to at least on a daily basis by the ExCo.</p>
        <p>The ExCo will arrange a meeting with you on Wednesday 22nd or Thursday 23rd, where you will be presenting how you developed the analysis, the main conclusions of it, and the outputs generated.</p>
        <p>Go for it!!!</p>
    </div>

    <div class="results-container">
        <div class="chart-container">
            {geo_map}
        </div>
        <div class="text-container">
        <h1>Global Investment Portfolio 3Q24</h1>

        <h2>North America</h2>
        <p>
            The <strong>USA</strong> remains the undisputed leader in the global investment portfolio, boasting a staggering <strong>€230.52M</strong> in investments, <strong>€293M</strong> in PnL, and a total NAV of <strong>€523M</strong>. With <strong>80 unique investments</strong>, the USA demonstrates a robust and mature investment ecosystem. The map’s darkest blue shade visually reinforces North America's dominance, particularly the USA's central role in driving the portfolio's success. This region is the backbone of the global portfolio, far outpacing contributions from other regions.
        </p>

        <h2>West Europe</h2>
        <p>
            <strong>Western Europe</strong> showcases diversity and steady growth, with contributions from key countries such as <strong>Spain</strong>, the <strong>UK</strong>, and <strong>Germany</strong>. The UK leads this regional performance with <strong>€18.58M</strong> in investments, followed by Spain at <strong>€4.94M</strong> and Germany at <strong>€0.55M</strong>. Spain also stands out with a significant <strong>52 unique investments</strong>, reflecting an active market. While the investment amounts are smaller than those of <strong>North America</strong>, Western Europe holds its strategic importance, as emphasized on the map with vibrant blue shades marking its key countries.
        </p>

        <h2>South America</h2>
        <p>
            <strong>Brazil</strong> anchors <strong>South America</strong> with exceptional performance, recording <strong>€64.88M</strong> in investments, <strong>€27.93M</strong> in PnL, and a NAV of <strong>€92.81M</strong>. With <strong>35 unique investments</strong>, Brazil underscores its position as a growing and vibrant market for venture capital. Other countries in the region, like <strong>Colombia</strong> and <strong>Chile</strong>, contribute more modestly, with investments of <strong>€0.24M</strong> and <strong>€0.44M</strong>, respectively. Although these amounts are smaller, South America continues to be a region of significance, driven by Brazil's strong performance and potential.
        </p>

        <h2>Israel</h2>
        <p>
            <strong>Israel</strong> emerges as an outlier with its unique and challenging position in the portfolio. Despite a substantial investment of <strong>€81.19M</strong>, Israel reports a concerning <strong>-€76.91M PnL</strong>, making it the only country in the portfolio with a negative PnL. Its total NAV of <strong>€4.27M</strong> remains overshadowed by this stark underperformance. This anomaly calls for a deeper analysis to identify the factors contributing to such a result and reassess strategies in this market. Israel's negative PnL stands out distinctly on the map, emphasizing the need for strategic intervention.
        </p>
        </div>
    </div>
    <div class="results-container">
        <div class="chart-container">
            {overall_nav_metrics}
        </div>
    <div class="text-container">
        <h1>NAV Metric Remarks per Quarter</h1>
    <h2>Investment and PnL</h2>
    <p>
        The chart provides a comparative analysis of <strong>Investment EUR</strong> and <strong>PnL EUR</strong> across four quarters, demonstrating consistent performance over time. 
        In <strong>4Q23</strong>, investments amounted to slightly above <strong>400M EUR</strong>, while profits reached just under <strong>500M EUR</strong>. 
        Moving to <strong>1Q24</strong>, a noticeable increase in both investments and profits can be observed, with investments climbing to approximately <strong>450M EUR</strong> and profits exceeding <strong>500M EUR</strong>.
    </p>
    <p>
        During <strong>2Q24</strong>, investment levels remained steady, matching those of <strong>1Q24</strong>, while PnL showed a slight decline, though still above <strong>500M EUR</strong>. 
        Finally, in <strong>3Q24</strong>, a minor decrease in both investments and profits is evident, with profits remaining robust near <strong>500M EUR</strong>.
    </p>

    <h2>Companies</h2>
    <p>
        Companies such as <strong>Sumup</strong>, <strong>Guideline</strong>, and <strong>Neon</strong> consistently dominate NAV weight, collectively accounting for over <strong>60%</strong> of the portfolio in most quarters. Smaller companies under "Other" also make up a significant share.
    </p>
    <h2>Investment Rounds</h2>
    <p>
        This chart illustrates the <strong>NAV distribution</strong> across various funding rounds over four quarters. The <strong>Series A</strong>, <strong>Series B</strong>, and <strong>Series C</strong> rounds clearly dominate the NAV weight consistently throughout all observed quarters. These rounds together represent the majority of NAV allocation, highlighting a strong emphasis on early to mid-stage investments.
    </p>
    <p>
        The <strong>Seed Stage</strong> and <strong>Series D</strong> rounds also make noticeable contributions, albeit smaller than the leading rounds. These contributions indicate a balanced strategy that includes investments in early and more mature stages of funding.
    </p>
    <h2>Funds</h2>
    <p>
        The <strong>Gospel US Fund</strong> contributes the largest NAV weight, consistently over <strong>30%</strong>, followed by the <strong>Gospel Global Fund</strong>, which contributes over <strong>25%</strong>. Other funds like the <strong>Gospel Brazil Fund (SPV)</strong> and <strong>Artemis BAVP</strong> make smaller contributions, typically below <strong>10%</strong>.
    </p>

    <h2>Sector</h2>
    <p>
        <strong>Financial Services</strong> maintain the highest NAV weight, consistently over <strong>50%</strong> across quarters. <strong>Business Services</strong> and <strong>Energy</strong> are the next largest contributors, each maintaining <strong>15-20% NAV weight</strong>. Sectors like <strong>Logistics & Mobility</strong> and <strong>Carbon Capture & Carbon Markets</strong> remain underrepresented, contributing less than <strong>5%</strong>.
    </p>
    <p>
    
    </p>
    </div>
    </div>
    
     <div class="results-container">
        <div class="chart-container">
            {fig_positive}
        </div>
            <div class="text-container">
<h2>Top Investment Performance Per Quarter</h2>
    <p> 
        <strong>Coinbase</strong> showcases consistent returns, achieving up to <strong>x104 fold</strong> during 1Q24 and maintaining strong results in other quarters with returns such as <strong>x88 fold</strong> in 2Q24 and <strong>x67 fold</strong> in 3Q24. 
        Furthermore, it is important to note, Coinbase is a listed company (NASDAQ: COIN).
    </p>
    <p>
        <strong>Sumup</strong>, managed by <strong>Gospel Legacy Portfolio</strong>, consistently delivers high returns across quarters, achieving returns like <strong>x13 fold</strong> in 2Q24 and 3Q24. 
    </p>
    <p>
        <strong>Grabango</strong>, managed by <strong>Gospel US Fund</strong>, maintains impressive returns, with a performance of <strong>x10 fold</strong> during 1Q24 and similar figures in subsequent quarters.
    </p>
    <p>
        <strong>Hippo (NYSE: HIPO)</strong>, another significant player managed by <strong>Gospel US Fund</strong>, showcases a steady return of <strong>x7 fold</strong> across quarters, emphasizing its consistent profitability. 
    </p>
        
    </div>
    </div>

     <div class="results-container">
        <div class="chart-container">
            {fig_negative}
        </div>
        <div class="text-container">
        <h2>Bottom Investment Performance Per Quarter</h2>
        <strong>Fourth Wall Climate Fund I Early-Stage; Flow Carbon</strong>, operating in the 
        <em>Carbon Capture & Carbon Markets</em> sector during 4Q23, experienced a severe loss with a percentage return of 
        <strong>-89.99%</strong>, significantly reducing its NAV compared to the investment.
    </p>
    <p>
        <strong>Gospel XYZ I; Ascendex SPV</strong>, part of the <em>Financial Services (Fintech, Insurtech, Crypto)</em> sector, 
        shows consistent negative performance across multiple quarters (1Q24, 2Q24, and 3Q24), with returns around 
        <strong>-89.99%</strong>, suggesting prolonged issues in profitability or valuation stability.
    </p>
    <p>
        <strong>Fourth Wall Climate Fund I Early-Stage; nZero</strong>, operating in the <em>Carbon Capture & Carbon Markets</em> 
        sector during 1Q24, reported a return of <strong>-84.81%</strong>, indicating a substantial devaluation from the initial investment.
    </p>
    <p>
        <strong>Fourth Wall Climate Fund I Early-Stage; Loop Global</strong>, part of the <em>RE & Construction</em> sector during 1Q24, 
        experienced a return of <strong>-82.53%</strong>, reflecting considerable losses within its sector.
    </p>
    <p>
        <strong>Fourth Wall Climate Fund I Early-Stage; Impulse</strong>, also part of the <em>RE & Construction</em> sector during 3Q24, 
        suffered the highest negative return in this list at <strong>-96.22%</strong>, nearly wiping out the NAV relative to the investment.
    </p>
        </div>
    </div>
    


    <div class="results-container">
        <div class="chart-container">
            {investment_overview}
        </div>
    <div class="text-container">
        <h2>Investment Type</h2>
    <h2>Investment Type Overview</h2>
    <p>
        The pie chart reveals that <strong>76.8%</strong> of the investments are categorized as <strong>First Investments</strong>, indicating a strong focus on initial capital deployment into new opportunities.
        <strong>18.5%</strong> of the portfolio is allocated to <strong>Follow-On Investments</strong>, showcasing a strategy to support existing portfolio companies in subsequent funding rounds.
    </p>
    <h2>Investment Instrument Overview</h2>
    <p>
        <strong>Equity Instruments lead</strong> the pack with a <strong>72.8%</strong> of the portfolio is allocation, emphasizing a focus on direct ownership and participation in company growth.
        While <strong>Debt-Like Instruments:</strong> represent <strong>14%</strong> of the investments such as convertible notes or bonds, offering a balance of downside protection and potential for equity conversion.
    </p>
    <p> Finally <strong> Other investments</strong> & <strong>Hybrid investment</strong> representing <strong>11%</strong> and <strong>2.42%</strong>, closed up the different investment instruments used for the portfolio creation.
    </div>
    </div>
    <div class="results-container">
        <div class="chart-container">
            {sector_manager_overview}
        </div>
        <div class="text-container">
        <h1>Management and Investment Distribution Insights</h1>
    <h2>Sector Distribution</h2>
    <p>
        <strong>Financial Services</strong> dominates the distribution, showcasing a strong focus on investments in fintech, insurtech, and crypto, which collectively lead the portfolio allocation.
        Following up: <strong> Energy and Business Services (IT, Productivity, CRM, HR)</strong> represent significant portions of the portfolio, reflecting strategic interest in sustainable energy and business optimization technologies.
    </p>
    <p>
        Finally, </strong> sectors like <strong>Healthtech</strong>, <strong>Circular Economy</strong>, and <strong>Mobility & Transportation</strong> are moderately represented, signaling a growing but cautious allocation to innovation-driven areas.
    </p>
    <p>
        <strong>Unfortunately, sectors:</strong> like <strong>Climate Software</strong>, <strong>Cybersecurity</strong>, and <strong>Foodtech</strong> are minimally represented, suggesting potential missed opportunities for diversification and growth.
    </p>

    <h2>Vertical Fund Distribution</h2>
    <p>
        <strong>Decarbonization</strong>dominates the portfolio, highlighting a strong commitment to sustainability and climate-related investments.
        <strong>Growth-oriented funds</strong> represent the next largest allocation, indicating a focus on scaling businesses with established traction and growth potential.
    </p>

    <h2>Management Distribution</h2>
    <p>
        <strong>Towercarbon</strong>leads the management distribution with a significant margin, indicating its pivotal role in managing a large portion of the portfolio.
        <strong>Gospel</strong> also represents a substantial share, reflecting a diversified approach with multiple fund managers contributing to the portfolio.
    </p>
    <p>
        Funds like <strong>Zumo Capital</strong> and <strong>Seawood</strong> are barely represented, suggesting limited engagement in portfolio management.
    </p>
        </div>
    </div>
    <div class="results-container">
        <div class="chart-container">
            {industry_overview}
        </div>
        <div class="text-container">
            <h2>Technology Overview</h2>
                <strong>AI & ML</strong> is the largest category, receiving 12.9% of the investments, highlighting its critical importance.
                Other significant categories include <strong>Data Analytics</strong> (10%) and <strong>Green Hydrogen</strong> (7.89%), reflecting strong trends in analytics and renewable energy technologies.
                A wide range of other technologies, such as <strong>Renewable Energy</strong>, <strong>FinTech</strong>, and <strong>Carbon Capture</strong>, are present but receive smaller proportions of investment.
            <h2>Customer Relationship Model Overview</h2>
                <strong>B2B</strong> is the predominant model, accounting for 61.4% of the share, emphasizing the importance of business-to-business relationships.
                <strong>B2C</strong> and <strong>B2B2C</strong> follow, contributing 22.8% and 9.58%, respectively.

            <h2>Business/Revenue Model Overview</h2>
                <strong>SaaS</strong> is the dominant model, representing 42.8%, underscoring its widespread adoption in generating revenue.
                <strong>Other</strong> (18.6%) and <strong>Marketplace</strong> (12.7%) models also hold significant shares, reflecting their relevance in modern digital platforms.
                Models, such as <strong>Free-Model</strong>, <strong>Licenses</strong>, and <strong>Subscription</strong>, make up smaller proportions, indicating their niche roles.
            <p>This distribution highlights a strong preference for scalable and recurring revenue models like SaaS and Marketplace.</p>
        </div>
    </div>

    <div class="results-container">
        <div class="chart-container">
            {differences_metrics_overview}
        </div>
        <div class="text-container">
            <h1>Investment and NAV Differences</h1>
            <h2>Distribution of Investment Differences (EUR)</h2>
   <p>
         The <strong> majority</strong> of investment differences cluster tightly around the <strong>0 EUR</strong> mark, indicating that most calculated investment values align closely with reported values.
         However, </strong> there are notable outliers exceeding <strong>±5M EUR</strong>, with a few extreme cases surpassing <strong>-10M EUR</strong>. These outliers could represent discrepancies in valuation calculations or reporting.
    </p>
    <h2>Distribution of NAV Differences (EUR)</h2>
    <p>
        Similar to investment differences, the <strong>majority</strong> of NAV differences also cluster around <strong>0 EUR</strong>, reflecting alignment between reported and calculated NAV values for most portfolio items.
        However, the presence of more pronounced outliers compared to investment differences is evident. Several points exceed <strong>±10M EUR</strong>, and a few fall below <strong>-15M EUR</strong>, signaling significant mismatches in NAV calculations or external factors affecting valuation.
    </p>
        </div>
    </div>
</body>
</html>
"""




# Save the HTML content to a file
html_file = pathlib.Path(r"C:\Users\juann\OneDrive\Escritorio\bvi.html")
html_file.write_text(html_content, encoding="utf-8")


436740